# DM Block Chain : Interaction avec la To Do App en utilisant Web3
## KARTOBI Sofiane  

### Objectif : Intéragir avec le smart contract de la to do application qui est déployé à travers MetaMask en utilisant le package Web3

#### Etape 0 : Installation et préparation
- Tout d'abord, on commence par installer le package Web3, j'ai suivi le tuto suivant expliquant cette démarche : https://youtu.be/DsAoff7yudQ?si=JN5Ktz5pTesvTb_p 
- Il s'est avéré qu'en suivant les étapes de ce tuto, l'installation n'a pas fonctionné (même en reforçant la version du json schema), je recevais un message d'erreur disant que le module "Crypto" est inexistant, je l'ai par la suite installé mais ça n'a pas aidé et le même probléme persisté (même en redémarrant la session). Finalement j'ai pu régler cela en changeant la version du kernel de anaconda à python 3.11.4

In [1]:
!pip install web3

  Using cached web3-6.11.4-py3-none-any.whl (1.6 MB)
                                              0.0/364.8 kB ? eta -:--:--
     ---                                      30.7/364.8 kB ? eta -:--:--
     ----                                  41.0/364.8 kB 487.6 kB/s eta 0:00:01
     -----------                            112.6/364.8 kB 1.1 MB/s eta 0:00:01
     --------------------                   194.6/364.8 kB 1.1 MB/s eta 0:00:01
     ----------------------------           276.5/364.8 kB 1.2 MB/s eta 0:00:01
     -------------------------------------  358.4/364.8 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 364.8/364.8 kB 1.3 MB/s eta 0:00:00
  Using cached eth_abi-4.2.1-py3-none-any.whl (28 kB)
  Using cached eth_account-0.10.0-py3-none-any.whl (109 kB)
  Using cached eth_hash-0.5.2-py3-none-any.whl (8.6 kB)
  Using cached eth_typing-3.5.2-py3-none-any.whl (14 kB)
  Using cached eth_utils-2.3.1-py3-none-any.whl (77 kB)
  Using cached hexbytes-0.3.1-py3-non


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install --force-reinstall jsonschema==3.2.0

  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
                                              0.0/63.3 kB ? eta -:--:--
     -------------------                      30.7/63.3 kB 1.4 MB/s eta 0:00:01
     ------------------------------------   61.4/63.3 kB 656.4 kB/s eta 0:00:01
     -------------------------------------- 63.3/63.3 kB 680.3 kB/s eta 0:00:00
  Using cached setuptools-69.0.2-py3-none-any.whl (819 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
web3 6.11.4 requires jsonschema>=4.0.0, but you have jsonschema 3.2.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from web3 import Web3

- On vérifie qu'on bien importé le package

In [7]:
Web3

web3.main.Web3

- J'ai déja déployé le contract sur remix avec mon compte MetaMask (différent de celui de Boutheina qu'on utilise d'habitude pour éviter tout conflit d'intéraction), d'ailleurs voici mon adresse : 0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2 
- Voici l'adresse du contract en question : 0x4b11999B58BCD87E666ED7fAdF207ae548E14A7c

- On peut également installer eth_tester pour tester une connexion standard à une blockchain fictive

In [4]:
!pip install eth_tester

                                              0.0/73.7 kB ? eta -:--:--
     ----------------                         30.7/73.7 kB ? eta -:--:--
     ----------------                         30.7/73.7 kB ? eta -:--:--
     ---------------------                  41.0/73.7 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 73.7/73.7 kB 449.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Etape 1 : Connexion au node et récupération du contract

- Nous allons nous connecter au node RPC de la blockchain Testnet BSC (chainID: 97), le lien est fourni à partir du site : https://docs.bscscan.com/misc-tools-and-utilities/public-rpc-nodes
- Par la suite, utiliser la methode HTTPProvider de Web3 et lui envoyer en paramétre cet url pour se connecter

In [8]:
# Connexion au node de chainIdD : 97 de bscscan test net
rpc_url = "https://data-seed-prebsc-1-s1.binance.org:8545/"
web3 = Web3(Web3.HTTPProvider(rpc_url))

In [9]:
web3

- Maintenant qu'on s'est connecté sur le RPC node, nous allons chercher notre contract déployé et le sauvegarder dans une variable pour pouvoir ensuite le manipuler et appeler ses fonctions.
- Pour se faire, nous avons besoin de l'adresse du contract déployé sur bnb test net et aussi de l'ABI du script après compilation. Ces paramétres seront directement extraits à partir de l'interface Remix puis on les passera à la methode concernée pour avoir le contract.

In [10]:
contract_address = "0x4b11999B58BCD87E666ED7fAdF207ae548E14A7c"

- Vu la structure de l'ABI les [  ] au debut et à la fin génaient qu'on j'avais créer la variable string avec double quotes (""), et donc il a fallu la mettre entre trois ''' (comme pour doc string)

In [11]:
contract_abi = '''[
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_task",
				"type": "string"
			},
			{
				"internalType": "uint256",
				"name": "_deadline",
				"type": "uint256"
			}
		],
		"name": "addTask",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_id",
				"type": "uint256"
			}
		],
		"name": "CheckOrUncheckTask",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_id",
				"type": "uint256"
			}
		],
		"name": "removeTask",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "taskId",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "taskDescription",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "deadline",
				"type": "uint256"
			}
		],
		"name": "TaskCreated",
		"type": "event"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "taskId",
				"type": "uint256"
			}
		],
		"name": "TaskRemoved",
		"type": "event"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "taskId",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "taskDescription",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "deadline",
				"type": "uint256"
			}
		],
		"name": "TaskUpdated",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_id",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_task",
				"type": "string"
			},
			{
				"internalType": "uint256",
				"name": "_deadline",
				"type": "uint256"
			}
		],
		"name": "updateTask",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "count",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getOverdueTodos",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "Task",
						"type": "string"
					},
					{
						"internalType": "bool",
						"name": "isCompleted",
						"type": "bool"
					},
					{
						"internalType": "uint256",
						"name": "deadline",
						"type": "uint256"
					}
				],
				"internalType": "struct ToDoList.TaskToDo[]",
				"name": "",
				"type": "tuple[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_id",
				"type": "uint256"
			}
		],
		"name": "getTask",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "Task",
						"type": "string"
					},
					{
						"internalType": "bool",
						"name": "isCompleted",
						"type": "bool"
					},
					{
						"internalType": "uint256",
						"name": "deadline",
						"type": "uint256"
					}
				],
				"internalType": "struct ToDoList.TaskToDo",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getUpcomingTodos",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "Task",
						"type": "string"
					},
					{
						"internalType": "bool",
						"name": "isCompleted",
						"type": "bool"
					},
					{
						"internalType": "uint256",
						"name": "deadline",
						"type": "uint256"
					}
				],
				"internalType": "struct ToDoList.TaskToDo[]",
				"name": "",
				"type": "tuple[]"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "List",
		"outputs": [
			{
				"internalType": "string",
				"name": "Task",
				"type": "string"
			},
			{
				"internalType": "bool",
				"name": "isCompleted",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "deadline",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]'''

In [12]:
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

In [13]:
contract

- A présent, nous avons acquérit notre contract ! 

#### Etape 2 : Intéraction et manipulation du contract en faisant appel aux fonctions définies

- Pour pouvoir utiliser les fonctions du contract, nous devons envoyer des transactions qui doivent être signées avec notre clé privée.
- J'ai déja extrait ma clé privé de mon compte MetaMask et je l'ai mise dans un fichier .txt que je vais exploiter pour extraire à nouveau cette clé dans mon code, sans la mettre explicitement dans mon code 

In [2]:
with open("MetaMaskPK.txt", "r") as f:
    private_key = f.read()

- Maintenant, je vais essayer d'appeler la fonction addTask() pour rajouter une nouvelle tache et vérifier par la suite si elle s'est bien executer et que la transaction est bien passée
- Nous devons d'abord créer la transaction en utilisant la methode buildtransaction, puis on l'a fait signer grace à notre clé privée et enfin on envoie la transaction signée grace à méthode sendRawTransaction
- Pour créer une tache, je dois non seulement lui donner un nom mais aussi rajouter sa deadline avec le format Unix Time Stamp en utilisant le site : https://www.unixtimestamp.com/ 
- Je vais ajouter comme tache ce DM actuel que je dois rendre avant minuit :) 

![Image](Capture.png)

- Afin d'envoyer la transaction, on peut spécifier explicitement l'adresse du compte from qui va envoyer cette transaction lorsqu'on créer la fonction dans l'argument "from" , comme on peut aussi setter le compte par défault à notre adresse sans avoir à spécifier ensuite cette adresse en argument grace à la méthode web3.eth.defaultAccount 

In [17]:
web3.eth.defaultAccount = web3.to_checksum_address("0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2")
task_name = "Envoyer le DM Web3"
task_deadline =1701989999

In [23]:
def addTask(task_name,task_deadline):

    # Construction de la transaction
    transaction = contract.functions.addTask(task_name,task_deadline).build_transaction(
        {
            "gasPrice": web3.eth.gas_price,
            "chainId": web3.eth.chain_id,
            "from": web3.to_checksum_address("0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2"),
            "nonce": web3.eth.get_transaction_count(web3.to_checksum_address("0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2"))
        }
    )

    # Signature de la transaction avec la clé privée
    signed_transaction = web3.eth.account.sign_transaction(transaction, private_key)

    # Envoi de la transaction signée et récupération du hachage de la transaction
    transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)
    transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash)
   
   
    return transaction_hash

- Finalement en omettant l'argument from j'ai reçu un message d'erreur disant que seul le owner du contract peut executer cette tache, alors que c'est la même adresse bizarrement, donc j'ai fini par rajouter cela explicitement

In [24]:
addTask(task_name=task_name,task_deadline=task_deadline)

HexBytes('0xe54f1fd1b8e3d27b466f92ce2dfe8011d0087824d9d24e841fee8f8763fa6061')

La transaction est belle et bien passée, on a obtenu son hash et on peut vérifier sur bnb test au niveau des transaction du contract qu'une transaction addTask s'est ajoutée et que son hash est identique à celui retourner par la fonction :

![Image](Capture2.png)

- Nous allons vérifier son contenu en appelant la fonction getTask(), ici nous n'aurons qu'à utiliser la methode call et non pas buildtransaction , étant donné que cette fonction getter n'est pas réellement une transaction lors de son appel

In [26]:
def getTask(task_id):
    return contract.functions.getTask(task_id).call()

In [27]:
getTask(0)

('Envoyer le DM Web3', False, 1701989999)

- Et voila l'appel a bien marché !
- Mais bon je ne sais pas lire la date format Unix Time Stamp donc je dois si je suis encore dans les temps pour cette tache ou si j'ai dépassé la deadline...

In [28]:
def getUpcomingTodos():
    return contract.functions.getUpcomingTodos().call()

In [29]:
getUpcomingTodos()

[('Envoyer le DM Web3', False, 1701989999)]

- Apparrement ça va, je suis encore on time :)

- Cependant, la tache est toujoutd false (pas encore cochée), étant donné que j'ai fini je vais créer la check grace à la fonction que je vais définir maintenant

In [30]:
def CheckOrUncheckTask(task_id):
    transaction = contract.functions.CheckOrUncheckTask(task_id).build_transaction(
        {
            "gasPrice": web3.eth.gas_price,
            "chainId": web3.eth.chain_id,
            "from": web3.to_checksum_address("0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2"),
            "nonce": web3.eth.get_transaction_count(web3.to_checksum_address("0xada2E183e53E1DDe87D4b40f1f4C2877481a1Ca2"))
        }
    )

    # Signature de la transaction avec la clé privée
    signed_transaction = web3.eth.account.sign_transaction(transaction, private_key)

    # Envoi de la transaction signée et récupération du hachage de la transaction
    transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)
    transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash)
   
   
    return transaction_hash

In [31]:
CheckOrUncheckTask(0)

HexBytes('0xab05c727a02b164be680ac2e232b01dc85254f38b2528ca1bc04150d9f6775f8')

- On va checker sur bnb testnet : 

![Image](Capture3.png)

Vérifions à présent si ça a bien été checké : 

In [32]:
getTask(0)

('Envoyer le DM Web3', True, 1701989999)

ET VOILA DONE !